<a href="https://colab.research.google.com/github/Mahalia-ym/entregaTFT/blob/main/python/K_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se importan las librerías necesarias para la red neuronal

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

Se da acceso al drive donde se encuentran los datos de los constructos que vamos a utilizar como input para la red neuronal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Aquí se cargan los valores de los constructos

In [ ]:
datosCompletos = pd.read_csv('/content/drive/My Drive/2º Semestre/TFT Nuevo/Red Neuronal/constructScores.csv', encoding='utf-8', sep=',', index_col=0)
datosCompletos.sample(10)


,Mindfulness,Satisf,Performance,Abandon
147,0.528684,-0.203327,1.105604,-0.266153
111,0.119059,0.282395,-0.952978,-1.405734
145,-0.244285,0.297158,0.316754,0.645001
161,1.124552,1.527304,1.280463,1.384622
87,0.323871,0.177505,-0.492484,-0.088135
132,1.040154,2.036459,0.798609,-1.405734
68,-1.573742,0.186176,-0.952978,-0.501475
153,-0.449097,-0.242008,-0.799480,-0.501475
31,0.122230,0.440515,-1.127836,-0.294805
24,-0.489036,-0.772308,-0.317626,1.086994


Establecemos la mejor estructura neuronal del anterior programa

In [ ]:
modelo=Sequential()
#Capa entrada
modelo.add(Dense(3, activation='relu', input_shape=(3,)))
#intermedias
modelo.add(Dense(8, activation='relu'))
modelo.add(Dense(3, activation='relu'))
#salida
modelo.add(Dense(1, activation='linear'))

In [ ]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 12        
                                                                 
 dense_1 (Dense)             (None, 8)                 32        
                                                                 
 dense_2 (Dense)             (None, 3)                 27        
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________


Establecemos dos variables las cuales sacarán los valores RMSE y R^2 de cada iteración según se indica en el KFold. Se dividirán en datos de entreno y prueba según la variable de salida (Abandon) o las variables de entrada (Mindfulness, Job Satisfaction y Performance)

In [ ]:
precision = []
precision2 = []
calculate_kf = KFold(n_splits=9, shuffle=False)
optimizador=tf.keras.optimizers.Adam(0.01)
modelo.compile(optimizer=optimizador, loss='mean_squared_error')
resultado= []
#Vamos a dividir los valores del kf
for valEntreno, valPrueba in calculate_kf.split(datosCompletos):
  entrenoX = datosCompletos.iloc[valEntreno].loc[:, ["Mindfulness", "Satisf", "Performance"]]
  pruebaX = datosCompletos.iloc[valPrueba].loc[:, ["Mindfulness", "Satisf", "Performance"]]
  entrenoY = datosCompletos.iloc[valEntreno].loc[:, ["Abandon"]]
  pruebaY = datosCompletos.iloc[valPrueba].loc[:, ["Abandon"]]
  modelo.fit(entrenoX, entrenoY, epochs=150, batch_size=100, verbose=0)
  pred_y = modelo.predict(pruebaX)
  #resultado = pred_y
  #print("mse")
  precision.append(mean_squared_error(pruebaY, pred_y, squared=False))
  #print(precision)

  #print("*********************")
  #print("R^2")
  precision2.append(r2_score(pruebaY, pred_y))
  #print(precision2)

print("mse")
print(precision)
print("***********************")
print("R^2")
print(precision2)


1/1 [==============================] - 0s 21ms/step
mse
[0.9442802219941956, 0.7402402987025967, 0.6130522148020157, 0.5046869710057655, 0.7092411418818237, 0.6868224814894647, 0.6134961477656179, 0.98893259896776, 0.7124950044293404]
***********************
R^2
[-0.07902267037340249, 0.37006428504056466, 0.5575968678891023, 0.6274757117837816, 0.6402355435773883, 0.6087561796197871, 0.5560201172640222, 0.07576629109243771, 0.3435329870217242]


Aquí tratamos de calcular el resultado promedio de las iteraciones anteriores según los RMSE y los R^2

In [ ]:
mse = 0
for i in precision:
  mse += i
print("Media mse:")
print(mse/9)


r2 = 0
for j in precision2:
  r2 += j
print("R^2:")
print(r2/9)

Media mse:
0.7236941201153978
R^2:
0.4111583681017117
